## Credit Curve Bootstrapper

The goal of the project is to build a bootstrapping procedure whose output will be a credit curve (an instance of the Python class CreditCurve). The input parameters to derive this curve are:

* The par rate of a set of Overnight Index Swaps (i.e. their market quotations)
* The par rate of a set of Credit Default Swap (i.e. their market quotations)

The resulting class will be used to compute the market value of a set of custom CDS.

Numerical results must be presented to the examining committee during a presentation in which the candidates will also explain the theoretical framework for CDS evaluation.

### Notes and hints

Numerical results must be presented to the examining committee during a presentation in which the candidates will also explain the theoretical framework for evaluation of risky flows.

All maturities are expressed in months and rates are expressed as fractions of one (i.e. 0.01 means 1%).

Remember to reuse the code we developed during the lessons in `finmarkets.py` as much as possible, instead of trying to rewrite everything from scratch!

In [1]:
from datetime import date

pricing_date = date(2013, 10, 31)

cds_recovery = 0.4

cds_quotes = [
    {'maturity': 12, 'spread':0.0149},
    {'maturity': 24, 'spread':0.0165},
    {'maturity': 36, 'spread':0.0173},
    {'maturity': 69, 'spread':0.0182},
    {'maturity': 120, 'spread':0.0183},
    {'maturity': 240, 'spread':0.0184},
]

ois_quotes = [
    {'maturity': 1, 'rate': 0.00106},
    {'maturity': 2, 'rate': 0.00114},
    {'maturity': 3, 'rate': 0.00115},
    {'maturity': 4, 'rate': 0.00117},
    {'maturity': 5, 'rate': 0.00119},
    {'maturity': 6, 'rate': 0.00121},
    {'maturity': 7, 'rate': 0.00122},
    {'maturity': 8, 'rate': 0.00124},
    {'maturity': 9, 'rate': 0.00128},
    {'maturity': 10, 'rate': 0.00131},
    {'maturity': 11, 'rate': 0.00135},
    {'maturity': 12, 'rate': 0.00138},
    {'maturity': 15, 'rate': 0.00152},
    {'maturity': 18, 'rate': 0.00166},
    {'maturity': 21, 'rate': 0.00184},
    {'maturity': 24, 'rate': 0.00206},
    {'maturity': 36, 'rate': 0.00344},
    {'maturity': 48, 'rate': 0.00543},
    {'maturity': 60, 'rate': 0.00756},
    {'maturity': 72, 'rate': 0.00967},
    {'maturity': 84, 'rate': 0.01162},
    {'maturity': 96, 'rate': 0.0134},
    {'maturity': 108, 'rate': 0.01502},
    {'maturity': 120, 'rate': 0.01649},
    {'maturity': 132, 'rate': 0.01776},
    {'maturity': 144, 'rate': 0.01888},
    {'maturity': 180, 'rate': 0.02137},
    {'maturity': 240, 'rate': 0.02322},
    {'maturity': 300, 'rate': 0.02389},
    {'maturity': 360, 'rate': 0.02416},
]

cds_to_price = [
    {'nominal': 5000000, 'maturity': 18, 'spread': 0.02},
    {'nominal': 5000000, 'maturity': 21, 'spread': 0.02},
    {'nominal': 5000000, 'maturity': 30, 'spread': 0.02},
    {'nominal': 5000000, 'maturity': 33, 'spread': 0.02},
    {'nominal': 5000000, 'maturity': 42, 'spread': 0.02},
    {'nominal': 5000000, 'maturity': 45, 'spread': 0.02},
    {'nominal': 5000000, 'maturity': 54, 'spread': 0.02},
    {'nominal': 5000000, 'maturity': 72, 'spread': 0.02},
    {'nominal': 5000000, 'maturity': 84, 'spread': 0.02},
    {'nominal': 5000000, 'maturity': 96, 'spread': 0.02},
    {'nominal': 5000000, 'maturity': 108, 'spread': 0.02},
    {'nominal': 5000000, 'maturity': 132, 'spread': 0.02},
    {'nominal': 5000000, 'maturity': 160, 'spread': 0.02},
    {'nominal': 5000000, 'maturity': 184, 'spread': 0.02},
    {'nominal': 5000000, 'maturity': 210, 'spread': 0.02},
]

In [2]:
from finmarkets import DiscountCurve, OvernightIndexSwap, generate_swap_dates

pillar_dates = [pricing_date]

swaps = []

for quote in ois_quotes:
    swap = OvernightIndexSwap(
        1,
        generate_swap_dates(
            pricing_date,
            quote['maturity']
        ),
        quote['rate'])
    
    swaps.append(swap)
    pillar_dates.append(swap.payment_dates[-1])
    
pillar_dates = sorted(pillar_dates)

In [3]:
def objective_function(x):
    
    curve = DiscountCurve(
        pricing_date, pillar_dates, x)
    
    sum_sq = 0.0
    
    for swap in swaps:
        sum_sq += swap.npv(curve) ** 2
        
    return sum_sq

In [4]:
from scipy.optimize import minimize

x0 = [1.0 for i in range(len(pillar_dates))]
bounds = [(0.01, 100.0) for i in range(len(pillar_dates))]
bounds[0] = (1.0, 1.0)

result = minimize(objective_function, x0, bounds=bounds)

In [5]:
print (result.x)
print (objective_function(x0))
print (objective_function(result.x))

[1.         0.99991167 0.99980687 0.99970619 0.99961015 0.99950111
 0.99939201 0.99928207 0.99916714 0.99903027 0.998895   0.99874906
 0.99858813 0.99807408 0.99748757 0.99674711 0.99582047 0.98957083
 0.97816267 0.9622282  0.94245084 0.91998041 0.89555722 0.86968693
 0.84284133 0.81600261 0.78929726 0.71347849 0.61197226 0.53249366
 0.46685426]
1.3988079511005296
1.116180166573695e-09


In [6]:
discount_curve = DiscountCurve(pricing_date, pillar_dates, result.x)

In [7]:
from finmarkets import generate_swap_dates, CreditDefaultSwap, CreditCurve

cds_pillar_dates = [pricing_date]

creditdefaultswaps = []
for quote in cds_quotes:
    creditdefswap = CreditDefaultSwap(
        1,
        pricing_date,
        quote['maturity']//12,
        quote['spread'])
    
    creditdefaultswaps.append(creditdefswap)
    cds_pillar_dates.append(creditdefswap.payment_dates[-1])
    
cds_pillar_dates = sorted(cds_pillar_dates)

In [8]:
def obj_function(unknown_ndps):

    curve_c = CreditCurve(cds_pillar_dates, unknown_ndps)
    sum_sq = 0.0

    for creditdefswap in creditdefaultswaps:
        sum_sq += creditdefswap.npv(discount_curve, curve_c) ** 2

    return sum_sq

In [9]:
x0_guess = [0.5 for i in range(len(cds_pillar_dates))]

bounds_credit_curve = [(0.01, 1) for i in range(len(cds_pillar_dates))]
bounds_credit_curve[0]=(1, 1)

results = minimize(obj_function, x0_guess, bounds=bounds_credit_curve)

In [10]:
print (results)
print (obj_function(x0_guess))
print (obj_function(results.x))

      fun: 8.833081717246685e-12
 hess_inv: <7x7 LbfgsInvHessProduct with dtype=float64>
      jac: array([-9.65377166e-09,  7.16687234e-07,  2.40422301e-06, -2.24757634e-06,
       -1.44850703e-06, -6.07960614e-07,  3.01896805e-07])
  message: b'CONVERGENCE: NORM_OF_PROJECTED_GRADIENT_<=_PGTOL'
     nfev: 88
      nit: 10
   status: 0
  success: True
        x: array([1.        , 0.90682878, 0.80418478, 0.70888378, 0.54482332,
       0.29728601, 0.08665697])
0.5418861901003296
8.833081717246685e-12


In [11]:
credit_curve = CreditCurve(cds_pillar_dates, results.x)

In [12]:
npv_cds_to_price = []

for quote in cds_to_price:
    creditdefswapprice = CreditDefaultSwap(
        quote['nominal'],
        pricing_date,
        quote['maturity'],
        quote['spread'])
    
    npv_cds_to_price.append(creditdefswapprice.npv(discount_curve, credit_curve))

print ("Prices of CDS " + str(["{:.2f}".format(p) for p in npv_cds_to_price]))

Prices of CDS ['-214372.45', '-238065.86', '-402085.09', '-450632.50', '-596274.73', '-644822.14', '-790464.37', '-1081748.84', '-1275938.48', '-1470128.12', '-1664317.77', '-2052697.05', '-2505806.22', '-2894185.51', '-3314929.73']
